In [ ]:
import json
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

#read training data
df_train = pd.read_json('project_training.json')
df_val = pd.read_json('project_validation.json') 

In [ ]:
df_train["climate"] = df_train["climate"].replace({"yes": 0, "no": 1})
df_train["sentiment"] = df_train["sentiment"].replace({"opportunity": 0, "neutral": 1, "risk": 2})
df_train["specificity"] = df_train["specificity"].replace({"spec": 0, "non": 1})
df_train["commitment"] = df_train["commitment"].replace({"yes": 0, "no": 1})
df_train

,text,climate,sentiment,commitment,specificity
0,The accelerator programs have sub-portfolios o...,0,1.0,1.0,1.0
1,"Also by means of BNDES Finem, we offer credit ...",1,NaN,NaN,NaN
2,Climate change Climate change exposes UPM to v...,0,2.0,1.0,1.0
3,Several tools and methodologies aimed at asses...,0,2.0,0.0,1.0
4,We worked with the UK government to accelerate...,0,0.0,0.0,0.0
...,...,...,...,...,...
395,"At the beginning of 2019, VINCI Airports signe...",1,NaN,NaN,NaN
396,We have also signed up to the Partnership for ...,0,2.0,0.0,1.0
397,Suzano also is involved and spearheads externa...,0,1.0,0.0,1.0
398,Risks to the Group’s reputation Risks include ...,1,NaN,NaN,NaN


In [ ]:
#read validation data
df_val["climate"] = df_val["climate"].replace({"yes": 0, "no": 1})
df_val["sentiment"] = df_val["sentiment"].replace({"opportunity": 0, "neutral": 1, "risk": 2})
df_val["specificity"] = df_val["specificity"].replace({"spec": 0, "non": 1})
df_val["commitment"] = df_val["commitment"].replace({"yes": 0, "no": 1})
df_val

,text,climate,sentiment,commitment,specificity
0,"Except as required by law, the Bank does not u...",1,NaN,NaN,NaN
1,"In terms of thermal coal, the Group has set an...",0,1.0,0.0,0.0
2,The engagement is driving improved conversatio...,0,1.0,0.0,1.0
3,The tool does this by tracking the generation ...,0,1.0,1.0,1.0
4,We also support and advise companies in the La...,0,0.0,0.0,0.0
...,...,...,...,...,...
395,"JetBlue drives safety management in many ways,...",1,NaN,NaN,NaN
396,10 Representative Concentration Pathways (RCP)...,0,1.0,1.0,1.0
397,Building on our history of energy efficiency i...,0,1.0,0.0,0.0
398,"In 2019, PME redesigned the passively managed ...",0,2.0,0.0,0.0


In [ ]:
#append data
df = df_train.append(df_val)

In [ ]:
df

,text,climate,sentiment,commitment,specificity
0,The accelerator programs have sub-portfolios o...,0,1.0,1.0,1.0
1,"Also by means of BNDES Finem, we offer credit ...",1,NaN,NaN,NaN
2,Climate change Climate change exposes UPM to v...,0,2.0,1.0,1.0
3,Several tools and methodologies aimed at asses...,0,2.0,0.0,1.0
4,We worked with the UK government to accelerate...,0,0.0,0.0,0.0
...,...,...,...,...,...
395,"JetBlue drives safety management in many ways,...",1,NaN,NaN,NaN
396,10 Representative Concentration Pathways (RCP)...,0,1.0,1.0,1.0
397,Building on our history of energy efficiency i...,0,1.0,0.0,0.0
398,"In 2019, PME redesigned the passively managed ...",0,2.0,0.0,0.0


In [ ]:
#climate label
df_train = df[["text", "climate"]]
df_train.rename(columns={"climate": "label"}, inplace = 1)
df_train

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,text,label
0,The accelerator programs have sub-portfolios o...,0
1,"Also by means of BNDES Finem, we offer credit ...",1
2,Climate change Climate change exposes UPM to v...,0
3,Several tools and methodologies aimed at asses...,0
4,We worked with the UK government to accelerate...,0
...,...,...
395,"JetBlue drives safety management in many ways,...",1
396,10 Representative Concentration Pathways (RCP)...,0
397,Building on our history of energy efficiency i...,0
398,"In 2019, PME redesigned the passively managed ...",0


In [ ]:
df_train.iloc[700]

text     In July 2020, BlackRock provided comments to t...
label                                                    0
Name: 700, dtype: object

In [ ]:
df_train.tail(10)

,text,label
790,We require all potential projects to be assess...,0
791,These sector-specific policies are usually dev...,1
792,CLIMATE SOLUTIONS change. “When we think about...,0
793,Climate change exacerbates existing risks in s...,0
794,Investec group including Investec Asset Manage...,0
795,"JetBlue drives safety management in many ways,...",1
796,10 Representative Concentration Pathways (RCP)...,0
797,Building on our history of energy efficiency i...,0
798,"In 2019, PME redesigned the passively managed ...",0
799,5.1.6 Commitment to monitor emissions from shi...,0


Best Model for climate: Bag of Words Stemming in Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

# Load the Reuters corpus from NLTK
corpus = nltk.corpus.reuters

# Define preprocessing function
stopwords = set(stopwords.words('english'))

def prep(t):
  l = t.split()
  #stem
  stemmer = PorterStemmer()
  l = [stemmer.stem(w) for w in l]
  l = [w for w in l if w not in stopwords and w.isalpha()]
  return ' '.join(l)

# Convert text to BoW representation
vectorizer = CountVectorizer(preprocessor=prep)
X = vectorizer.fit_transform(df_train['text'])
y = df_train['label']


# Generate list for storing results
results = []
accuracy = []

# Define Logistic regression model
model = LogisticRegression(penalty = 'none')

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    #print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_climate = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_climate = pd.DataFrame(result_climate).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_climate.to_excel(writer, sheet_name="climate_LR")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

           0       0.90      0.98      0.93       131
           1       0.82      0.48      0.61        29

    accuracy                           0.89       160
   macro avg       0.86      0.73      0.77       160
weighted avg       0.88      0.89      0.88       160

(0.8593171534348005, 0.7299289286654382, 0.7715011107584894, None)
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       138
           1       0.68      0.59      0.63        22

    accuracy                           0.91       160
   macro avg       0.81      0.77      0.79       160
weighted avg       0.90      0.91      0.90       160

(0.8101903695408734, 0.7737154150197629, 0.7901914503015999, None)
              precision    recall  f1-score   support

           0       0.89      0.92      0.91       126
           1       0.67      0.59      0.62        34

    accuracy                           0.85     

In [ ]:
#sentiment
df_train_s = df[["text", "sentiment"]].dropna().reset_index(drop=True)
df_train_s.rename(columns={"sentiment": "label"}, inplace = 1)
df_train_s["label"] = df_train_s["label"].astype(int)
df_train_s

,text,label
0,The accelerator programs have sub-portfolios o...,1
1,Climate change Climate change exposes UPM to v...,2
2,Several tools and methodologies aimed at asses...,2
3,We worked with the UK government to accelerate...,0
4,"(changes in consumption behavior, failure of i...",2
...,...,...
656,Investec group including Investec Asset Manage...,2
657,10 Representative Concentration Pathways (RCP)...,1
658,Building on our history of energy efficiency i...,1
659,"In 2019, PME redesigned the passively managed ...",2


Best model for sentiment: TF-IDF ngram_range=(1,2) smooth_idf = True in ridge

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(preprocessor=prep, ngram_range = (1, 2), smooth_idf = True)

X_s = tf.fit_transform(df_train_s['text'])
y_s = df_train_s['label']


results = []
accuracy = []

# Define Ridge regression model
model = LogisticRegression(penalty='l2', C=1.0)

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_s):
    X_train, X_val = X_s[train_index], X_s[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_s.iloc[train_index], y_s.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_sentiment = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_sentiment = pd.DataFrame(result_sentiment).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_sentiment.to_excel(writer, sheet_name="sentiment_ridge")

[1 2 2 2 0 1 2 1 2 1 2 1 1 0 1 2 1 2 1 1 1 1 2 1 1 1 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 1 2 0 1 2 1 2 1 0 1 1 1 1 1 2 1 2 1 1 2 2 0 1 2 1 2 0 1 2 1 2 2 0 1
 2 1 1 1 1 2 1 1 1 2 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 2 1 1 2 1 1 2 1 1
 1 1 1 1 2 1 2 1 0 0 2 1 1 1 2 1 2 2 1 2 1 1]
              precision    recall  f1-score   support

           0       0.87      0.39      0.54        33
           1       0.65      0.92      0.76        60
           2       0.88      0.75      0.81        40

    accuracy                           0.74       133
   macro avg       0.80      0.69      0.71       133
weighted avg       0.78      0.74      0.72       133

(0.8012605042016806, 0.6868686868686869, 0.7054554554554553, None)
[1 2 2 1 0 1 1 1 2 0 1 1 1 0 2 0 1 1 2 1 2 1 2 2 2 1 1 1 1 2 1 1 1 1 1 1 2
 1 2 1 1 1 1 2 1 1 1 1 2 2 2 1 1 0 1 1 2 1 1 2 0 1 0 1 1 1 2 1 2 1 1 1 0 2
 0 1 1 2 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 2 1 1 1 1 2 2 1 2 1 1 2 1 2 0 1
 1 1 2 0 1 1 0 1 1 1 1 1 1 1 1 2 1 1 1 2 1]
            

Best Model for sentiment: Bag of Words lemmatization ridge

In [ ]:
import nltk
nltk.download('omw-1.4')
  
#preprocessing
def prep(t):
  l = t.split()
  #lemmatize
  lemmatizer = WordNetLemmatizer()
  l = [lemmatizer.lemmatize(w) for w in l]
  l = [w for w in l if w not in stopwords]
  l = [w for w in l if w.isalpha()]
  return ' '.join(l)

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(preprocessor=prep, ngram_range = (1, 2), smooth_idf = True)

X_s = tf.fit_transform(df_train_s['text'])
y_s = df_train_s['label']


results = []
accuracy = []

# Define Ridge regression model
model = LogisticRegression(penalty='l2', C=1.0)

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_s):
    X_train, X_val = X_s[train_index], X_s[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_s.iloc[train_index], y_s.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_sentiment_lemma = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_sentiment_lemma = pd.DataFrame(result_sentiment_lemma).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_sentiment_lemma.to_excel(writer, sheet_name="sentiment_ridge_lemma")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[1 2 2 2 0 1 2 1 2 2 2 1 1 1 1 2 1 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 2 0 1 2 1 2 1 0 1 1 1 1 1 2 1 2 1 1 2 1 1 1 2 1 2 1 1 2 1 2 2 0 1
 2 1 1 1 1 2 1 1 1 2 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 0 1 2 1 1 2 1 1
 1 1 1 1 2 1 2 1 0 0 2 1 1 1 2 1 2 2 1 2 1 1]
              precision    recall  f1-score   support

           0       1.00      0.27      0.43        33
           1       0.63      0.93      0.75        60
           2       0.83      0.72      0.77        40

    accuracy                           0.71       133
   macro avg       0.82      0.64      0.65       133
weighted avg       0.78      0.71      0.68       133

(0.8192616372391653, 0.6436868686868688, 0.6511942047512518, None)
[1 2 2 1 1 1 1 1 2 1 1 1 1 0 2 1 1 1 2 1 2 1 2 2 2 1 1 1 1 2 1 1 1 1 1 1 2
 1 2 1 1 1 1 2 1 1 1 1 2 2 2 1 1 0 1 1 2 1 1 2 0 1 0 1 1 0 2 1 2 1 1 1 1 2
 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 2 1 1 1 1 2 2 1 2 1 1 2 1 2 0 1
 1 1 2 0 1 1 0 1 1 1 1 2 1 1 2 2 1 1 1 2 1]
            

Best model for sentiment: Bag of Words lemmatization balanced ridge

In [ ]:
import nltk
nltk.download('omw-1.4')
  
#preprocessing
def prep(t):
  l = t.split()
  #lemmatize
  lemmatizer = WordNetLemmatizer()
  l = [lemmatizer.lemmatize(w) for w in l]
  l = [w for w in l if w not in stopwords]
  l = [w for w in l if w.isalpha()]
  return ' '.join(l)

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(preprocessor=prep, ngram_range = (1, 2), smooth_idf = True)

X_s = tf.fit_transform(df_train_s['text'])
y_s = df_train_s['label']


results = []
accuracy = []

# Define Ridge regression model
model = LogisticRegression(penalty='l2', C=1.0, class_weight = 'balanced')

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_s):
    X_train, X_val = X_s[train_index], X_s[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_s.iloc[train_index], y_s.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_sentiment_balanced = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_sentiment_balanced = pd.DataFrame(result_sentiment_balanced).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_sentiment_balanced.to_excel(writer, sheet_name="sentiment_ridge_balanced")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[1 2 2 2 0 1 2 1 2 2 2 1 1 0 1 2 1 2 1 1 1 1 2 1 0 1 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 1 2 0 0 2 1 2 1 0 1 0 1 1 1 2 1 2 0 1 2 2 0 1 2 1 2 0 1 2 1 2 2 0 1
 2 1 1 1 1 2 2 1 1 2 0 1 0 1 2 1 1 1 1 1 0 1 1 1 1 0 0 2 2 0 1 2 0 1 2 1 0
 1 1 1 1 2 1 2 1 0 0 2 1 1 1 2 0 2 2 0 2 1 1]
              precision    recall  f1-score   support

           0       0.80      0.61      0.69        33
           1       0.71      0.83      0.77        60
           2       0.84      0.80      0.82        40

    accuracy                           0.77       133
   macro avg       0.79      0.75      0.76       133
weighted avg       0.77      0.77      0.76       133

(0.7854636591478696, 0.7464646464646464, 0.7597995873857943, None)
[1 2 2 1 0 1 0 1 2 0 1 1 1 0 2 0 1 1 2 0 2 1 2 2 2 0 1 1 1 2 0 0 0 1 1 1 2
 1 2 1 1 1 1 2 1 1 0 0 2 2 2 1 1 0 1 1 2 1 1 2 0 1 0 1 0 0 2 1 2 1 1 1 0 2
 0 0 1 2 1 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 2 1 1 1 1 2 2 1 2 2 1 2 0 2 0 1
 1 1 2 0 1 1 0 2 0 1 2 2 1 2 2 2 1 1 1 2 1]
            

Best Model for commitment: TF-IDF stemming ridge

In [ ]:
#commitment
df_train_c = df[["text", "commitment"]].dropna().reset_index(drop=True)
df_train_c.rename(columns={"commitment": "label"}, inplace = 1)
df_train_c["label"] = df_train_c["label"].astype(int)
df_train_c

,text,label
0,The accelerator programs have sub-portfolios o...,1
1,Climate change Climate change exposes UPM to v...,1
2,Several tools and methodologies aimed at asses...,0
3,We worked with the UK government to accelerate...,0
4,"(changes in consumption behavior, failure of i...",1
...,...,...
656,Investec group including Investec Asset Manage...,1
657,10 Representative Concentration Pathways (RCP)...,1
658,Building on our history of energy efficiency i...,0
659,"In 2019, PME redesigned the passively managed ...",0


In [ ]:
tf = TfidfVectorizer(preprocessor=prep)

X_c = tf.fit_transform(df_train_c['text'])
y_c = df_train_c['label']

results = []
accuracy = []

# Define Ridge regression model
model = LogisticRegression(penalty='l2', C=1.0)

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_c):
    X_train, X_val = X_c[train_index], X_c[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_c.iloc[train_index], y_c.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))
    #accuracy.append(accuracy_score(y_val, score))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_commitment = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_commitment = pd.DataFrame(result_commitment).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_commitment.to_excel(writer, sheet_name="commitment_ridge")

[0 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1
 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 1 1 0 1
 1 1 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 1 1 0
 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1]
              precision    recall  f1-score   support

           0       0.82      0.69      0.75        61
           1       0.77      0.88      0.82        72

    accuracy                           0.79       133
   macro avg       0.80      0.78      0.78       133
weighted avg       0.79      0.79      0.79       133

(0.7959110473457676, 0.7817622950819672, 0.784090909090909, None)
[1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1
 1 1 0 1 1 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1
 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0]
              precision    recall  f1-score   support

           0

Best model for commitment: TF-IDF stemming svm

In [ ]:
tf = TfidfVectorizer(preprocessor=prep)

X_c = tf.fit_transform(df_train_c['text'])
y_c = df_train_c['label']

results = []
accuracy = []

# Define Ridge regression model
model = SVC(kernel='rbf', C=1.0)

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_c):
    X_train, X_val = X_c[train_index], X_c[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_c.iloc[train_index], y_c.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))
    #accuracy.append(accuracy_score(y_val, score))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_commitment_svm = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_commitment_svm = pd.DataFrame(result_commitment_svm).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_commitment_svm.to_excel(writer, sheet_name="commitment_svm")

[0 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1
 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1
 1 1 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 1 1 0
 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1]
              precision    recall  f1-score   support

           0       0.84      0.70      0.77        61
           1       0.78      0.89      0.83        72

    accuracy                           0.80       133
   macro avg       0.81      0.80      0.80       133
weighted avg       0.81      0.80      0.80       133

(0.8118125298900047, 0.7969034608378871, 0.7995129870129871, None)
[1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1
 1 1 0 1 1 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 0 1 1 0 1
 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0]
              precision    recall  f1-score   support

           

In [ ]:
#specificity
df_train_sp = df[["text", "specificity"]].dropna().reset_index(drop=True)
df_train_sp.rename(columns={"specificity": "label"}, inplace = 1)
df_train_sp["label"] = df_train_sp["label"].astype(int)
df_train_sp

,text,label
0,The accelerator programs have sub-portfolios o...,1
1,Climate change Climate change exposes UPM to v...,1
2,Several tools and methodologies aimed at asses...,1
3,We worked with the UK government to accelerate...,0
4,"(changes in consumption behavior, failure of i...",1
...,...,...
656,Investec group including Investec Asset Manage...,1
657,10 Representative Concentration Pathways (RCP)...,1
658,Building on our history of energy efficiency i...,0
659,"In 2019, PME redesigned the passively managed ...",0


Best Model for specificity: TF-IDF ngram_range=(1,2) smooth_idf = True in ridge

In [ ]:
tf = TfidfVectorizer(preprocessor=prep, ngram_range = (1, 2), smooth_idf = True)

X_sp = tf.fit_transform(df_train_sp['text'])
y_sp = df_train_sp['label']

results = []
accuracy = []

# Define Ridge regression model
model = LogisticRegression(penalty='l2', C=1.0)

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_sp):
    X_train, X_val = X_sp[train_index], X_sp[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_sp.iloc[train_index], y_sp.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_specificity = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_specificity = pd.DataFrame(result_specificity).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_specificity.to_excel(writer, sheet_name="specificity_ridge")

[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1]
              precision    recall  f1-score   support

           0       0.94      0.31      0.47        54
           1       0.68      0.99      0.80        79

    accuracy                           0.71       133
   macro avg       0.81      0.65      0.64       133
weighted avg       0.79      0.71      0.67       133

(0.8113526570048308, 0.6510782934833568, 0.6381729667812142, None)
[1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           

Best Model for specifity: TF-IDF stemming in svm

In [ ]:
tf = TfidfVectorizer(preprocessor=prep)

X_sp = tf.fit_transform(df_train_sp['text'])
y_sp = df_train_sp['label']

results = []
accuracy = []

# Define SVM model
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1.0)

# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
scores = []
for train_index, val_index in kf.split(X_sp):
    X_train, X_val = X_sp[train_index], X_sp[val_index]
    #y_train, y_val = y[train_index], y[val_index]
    y_train, y_val = y_sp.iloc[train_index], y_sp.iloc[val_index]

    model.fit(X_train, y_train)
    score = model.predict(X_val)

    print(score)
    print(classification_report(y_val, score))

    print(precision_recall_fscore_support(y_val, score, average='macro'))

    results.append(precision_recall_fscore_support(y_val, score, average='macro'))
    accuracy.append(accuracy_score(y_val, score))

# Compute mean and standard deviation of cross-validation scores
avg_precision = np.mean([results[0][0], results[1][0], results[2][0], results[3][0], results[4][0]])
avg_recall = np.mean([results[0][1], results[1][1], results[2][1], results[3][1], results[4][1]])
avg_f = np.mean([results[0][2], results[1][2], results[2][2], results[3][2], results[4][2]])

print(f"average precision: {avg_precision}")
print(f"average recall: {avg_recall}")
print(f"average f1: {avg_f}")
print(f"average accuracy: {np.mean(accuracy)}")

result_specificity_svm = {"macro avg":{"average precision" : avg_precision,
                               "average recall" : avg_recall,
                               "average f1" : avg_f,
                               "average accuracy" : np.mean(accuracy)
                               }}
result_specificity_svm = pd.DataFrame(result_specificity_svm).transpose()
with pd.ExcelWriter("metrics_new.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    result_specificity_svm.to_excel(writer, sheet_name="specificity_svm")

[1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1
 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0
 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1]
              precision    recall  f1-score   support

           0       0.88      0.52      0.65        54
           1       0.74      0.95      0.83        79

    accuracy                           0.77       133
   macro avg       0.81      0.73      0.74       133
weighted avg       0.80      0.77      0.76       133

(0.8087871287128713, 0.7339428035630566, 0.7422480620155039, None)
[1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1
 0 1 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1
 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0
 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0]
              precision    recall  f1-score   support

           